<a href="https://colab.research.google.com/github/nsrnkha/PCD_2006154/blob/main/Image_clasification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split

import tensorflow as tf

In [ ]:
image_dir = Path('../input/basicshapes/shapes/shapes')

# Creating File DataFrame

In [ ]:
filepaths = list(image_dir.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

image_df = pd.concat([filepaths, labels], axis=1)

train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

In [ ]:
train_df

,Filepath,Label
253,../input/basicshapes/shapes/shapes/squares/dra...,squares
19,../input/basicshapes/shapes/shapes/circles/dra...,circles
14,../input/basicshapes/shapes/shapes/circles/dra...,circles
91,../input/basicshapes/shapes/shapes/circles/dra...,circles
296,../input/basicshapes/shapes/shapes/squares/dra...,squares
...,...,...
203,../input/basicshapes/shapes/shapes/squares/dra...,squares
255,../input/basicshapes/shapes/shapes/squares/dra...,squares
72,../input/basicshapes/shapes/shapes/circles/dra...,circles
235,../input/basicshapes/shapes/shapes/squares/dra...,squares


In [ ]:
test_df

,Filepath,Label
189,../input/basicshapes/shapes/shapes/triangles/d...,triangles
123,../input/basicshapes/shapes/shapes/triangles/d...,triangles
185,../input/basicshapes/shapes/shapes/triangles/d...,triangles
213,../input/basicshapes/shapes/shapes/squares/dra...,squares
106,../input/basicshapes/shapes/shapes/triangles/d...,triangles
...,...,...
181,../input/basicshapes/shapes/shapes/triangles/d...,triangles
290,../input/basicshapes/shapes/shapes/squares/dra...,squares
244,../input/basicshapes/shapes/shapes/squares/dra...,squares
197,../input/basicshapes/shapes/shapes/triangles/d...,triangles


# Loading Image Data

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

Found 168 validated image filenames belonging to 3 classes.
Found 42 validated image filenames belonging to 3 classes.
Found 90 validated image filenames belonging to 3 classes.


# Training

In [ ]:
inputs = tf.keras.Input(shape=(28, 28, 1))
x = tf.keras.layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint('./model.h5', save_best_only=True, save_weights_only=True)
    ]
)

Epoch 1/100
6/6 [==============================] - 2s 207ms/step - loss: 1.0995 - accuracy: 0.3223 - val_loss: 1.1016 - val_accuracy: 0.3333
Epoch 2/100
6/6 [==============================] - 0s 50ms/step - loss: 1.0939 - accuracy: 0.3828 - val_loss: 1.1011 - val_accuracy: 0.3333
Epoch 3/100
6/6 [==============================] - 0s 50ms/step - loss: 1.0972 - accuracy: 0.3410 - val_loss: 1.0998 - val_accuracy: 0.2619
Epoch 4/100
6/6 [==============================] - 0s 52ms/step - loss: 1.0991 - accuracy: 0.3305 - val_loss: 1.0979 - val_accuracy: 0.3333
Epoch 5/100
6/6 [==============================] - 0s 55ms/step - loss: 1.0971 - accuracy: 0.4012 - val_loss: 1.0990 - val_accuracy: 0.3810
Epoch 6/100
6/6 [==============================] - 0s 52ms/step - loss: 1.0977 - accuracy: 0.4832 - val_loss: 1.1007 - val_accuracy: 0.5476
Epoch 7/100
6/6 [==============================] - 0s 55ms/step - loss: 1.0961 - accuracy: 0.4034 - val_loss: 1.1085 - val_accuracy: 0.2619
Epoch 8/100
6/6 [==

In [ ]:
model.load_weights('./model.h5')

# Results

In [ ]:
results = model.evaluate(test_images, verbose=0)

print("    Loss: {:.4f}".format(results[0]))
print("Accuracy: {:.2f}%".format(results[1] * 100))

    Loss: 0.5284
Accuracy: 78.89%
